In [114]:
# ymyD7ijqAGHxBJYOpjrHZtlC2mHZYEN6
# ygqDyviPRT7ZOkLP

from amadeus import Client, ResponseError
import requests
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import numpy as np
import sqlite3
import json
import traceback
from sqlite3 import IntegrityError
import csv
import datetime

amadeus = Client(
    client_id='ymyD7ijqAGHxBJYOpjrHZtlC2mHZYEN6',
    client_secret='ygqDyviPRT7ZOkLP'
)


In [71]:
def create_connection(db_file="event_travel.db"):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [72]:
def execute_select_query(q,cur,conn):
    try:
        cur.execute(q)
    except IntegrityError:
        print('Found a duplicate already in the DB.')
#         traceback.print_exc()   

def execute_insert_query(q,val,cur,conn):
    try:
        cur.execute(q, val)
        conn.commit()
    except IntegrityError:
        print('Found a duplicate already in the DB.')
#         traceback.print_exc()    


In [73]:
def get_distance(lat1, lon1, lat2, lon2):
    
    # approximate radius of earth in km
    R = 6373.0
    Miles_per_Km = 0.621371

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c * Miles_per_Km

    return distance

In [74]:
def find_closest_airports(lat, long, r):
    ret = []
    best = r
    
    conn = create_connection()
    cur = conn.cursor()
    execute_select_query(f'select * from airports',cur,conn)
    airports = cur.fetchall()
    conn.close()

    for airport in airports:

        distance = get_distance(lat, long, airport[4], airport[5])
        if distance <= r:
            data = {'id':airport[0],'name':airport[1],'distance':distance,'airport':airport[3]}
            if distance < best:
                ret.insert(0,data)
                best = distance
            else:
                ret.append(data)
                
    
    return sorted(ret, key=lambda k: k['distance'])             

In [75]:
where_are_you = 'SLC'

In [96]:
likely_departure_points = [
    [40.790152, -111.979038, 'Salt Lake City', 'UT'],
     [36.085044, -115.149928, 'Las Vegas', 'NV'],
     [39.851704, -104.673811, 'Denver', 'CO']
]
for dept in likely_departure_points:
    print(dept)
    ret = find_closest_airports(dept[0],dept[1],30)
    print(len(ret))
    for x in ret:
        print(x)

[40.790152, -111.979038, 'Salt Lake City', 'UT']
1
{'id': '3536', 'name': 'Salt Lake City International Airport', 'distance': 0.13286185225979918, 'airport': 'SLC'}
[36.085044, -115.149928, 'Las Vegas', 'NV']
1
{'id': '3877', 'name': 'McCarran International Airport', 'distance': 0.360713200823457, 'airport': 'LAS'}
[39.851704, -104.673811, 'Denver', 'CO']
1
{'id': '3751', 'name': 'Denver International Airport', 'distance': 0.6920960602071604, 'airport': 'DEN'}


In [77]:
def extract_event_json(event):
    try:
        name = event['name']
        event_id = event['id']

        venue = event['_embedded']['venues'][0]
        location = {'address':venue['address']['line1'],'city':venue['city']['name'],'state':venue['state']['stateCode'],
                'latlong':[float(venue['location']['latitude']),float(venue['location']['longitude'])]}

        latlong = location['latlong']
        airports = find_closest_airports(latlong[0], latlong[1], 20)
        event['airports'] = airports
    except KeyError:
        print('KeyError')
#         traceback.print_exc()
        event = {}

    return event


In [80]:
def find_events_by_classification(page_number=0, classification='music'):
    remaining_pages = True
    total_pages = 0
    page_count = 0
    ret = []
    conn = create_connection()
    cur = conn.cursor()
    while remaining_pages:
        try:
            URL = f"https://app.ticketmaster.com/discovery/v2/events.json"
            PARAMS = {'apikey':'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page':page_number, 
                      'countryCode':'US', 'classificationName':classification,'size':25} 
            print(PARAMS)
            r = requests.get(url = URL, params = PARAMS) 
            data = r.json()
            page = data['page']
            size = page['size']
            page_number = page['number']
            total_pages = page['totalPages']
            events = data['_embedded']['events']
            print(f'page: {page_number}/{total_pages}')
            #         print(len(events))
            for event in events:
                event_json = extract_event_json(event)
                if len(event_json) > 0:
                    name = event_json['name']
                    if 'priceRanges' in event_json:
            #             print(name)
                        line = (event_json['id'], name, classification, json.dumps(event_json))
                        execute_insert_query('insert into raw_events(event_id, name, classification, json) values(?,?,?,?)', line,cur,conn)
        except KeyError:
            traceback.print_exc()
        except IntegrityError:
            traceback.print_exc()

        remaining_pages = ( page_number * size ) < 300
        page_number += 1
        
    conn.close()

 

In [97]:
find_events_by_classification(0,'music')

{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 0, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 0/741
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 1, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 1/741
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 2, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 2/741
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 3, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 3/741
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 4, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 4/741
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the D

In [98]:
def get_event_names(from_raw=True):
    conn = create_connection()
    cur = conn.cursor()
    if from_raw:
        cur.execute(f'select distinct name from raw_events order by name')
    else:
        cur.execute(f'select distinct name from my_events order by name')
    names = cur.fetchall()
    conn.close()
    for name in names:
        print(name[0])

In [99]:
get_event_names()

2 Day Pass: Stick Figure - Once in a Lifetime Tour ft. Collie Buddz, The Movement & Iya Terra
2020 B105 Country Megaticket
2020 Country Megaticket
2020 Jones Beach Country Megaticket
2020 KSON Country Megaticket
2020 Pnc Bank Arts Center Country Megaticket Presented By Pennzoil
A Thousand Thoughts: A live documentary with the Kronos Quartet
ALL THAT REMAINS w/ Butcher Babies, Saul
Bikini Kill with Hurry Up
Billy Joel
Buddy Guy with Jonny Lang
Chris Stapleton's All-American Roadshow
Chris Stapleton: Concert for Kentucky, An Outlaw State of Kind Benefit
Concert for Peace
Country 92-5 Megaticket
Country Megaticket
Country Megaticket 2020
Dave Matthews Band
David Archuleta
Def Leppard/Mötley Crüe/Poison/Joan Jett and the Blackhearts
Feed The Streetz Tour 2020
Florida Chamber Presents: Joseph Haydn, Caroline Shaw & Beethoven
Frankenstein (1931)
Freshwater Land Trust's Land Aid featuring Umphrey's McGee
Garth Brooks
George Strait
Grease Sing-Along
Hammerfall, Beast in Black, Edge of Paradise

In [107]:
def store_event(event, my_location):
    conn = create_connection()
    cur = conn.cursor()
    
    dates = event['dates']['start']
    event_id = event['id']
    sales = event['sales']['public']

    prices = event['priceRanges']
    venue = event['_embedded']['venues'][0]
    location = {'address':venue['address']['line1'],'city':venue['city']['name'],'state':venue['state']['stateCode'],
                'latlong':[float(venue['location']['latitude']),float(venue['location']['longitude'])]}

    event_date = dates['localDate']
    event_time = dates['localTime']
    price_range = [prices[0]['min'],prices[0]['max']]
    distance = get_distance(my_location[0],my_location[1],location['latlong'][0],location['latlong'][1])
    latlong = location['latlong']
    airports = find_closest_airports(latlong[0], latlong[1], 20)

    if len(airports)>0:
        val = (event_id,
               event['name'],
               event_date,
               event_time,
               price_range[0],
               price_range[1],
               location['address'],
               location['city'],
               location['state'],
               latlong[0],
               latlong[1],
               distance,
               my_location[0],
               my_location[1],
               my_location[2],
               my_location[3],
               my_location[4],
               airports[0]['airport']
              )

        execute_insert_query("""INSERT INTO my_events(
        id,name,date,time,price_low,price_high,street_address,
        city,state,lat,long,distance,my_lat,my_long,my_city,my_state,depart_airport,dest_airport) 
        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",val,cur,conn)

#     for airport in airports:
#         val2 = (event_id ,airport['airport'],airport['distance'])
#         execute_insert_query('insert into airports(event_id, airport, distance) values(?,?,?)',val2,cur,conn)

    conn.close()


In [108]:
def select_events(keyword, my_location):
    conn = create_connection()
    cur = conn.cursor()
    cur.execute(f'select * from raw_events where name like "%{keyword}%"')
    events = cur.fetchall()
    conn.close()
    
    for event in events:
        name = event[1]
#         print(name)
        event_json = json.loads(event[3])
        store_event(event_json, my_location)    

In [109]:
preferred_event_names = [
    'Harry Styles']

likely_departure_points = [
    [40.790152, -111.979038, 'Salt Lake City', 'UT', 'SLC']]
#     [36.085044, -115.149928, 'Las Vegas', 'NV'],
#     [39.851704, -104.673811, 'Denver', 'CO']]

for event_name in preferred_event_names:
    for location in likely_departure_points:
        print(f'Getting {event_name} from {location[2]}')
        select_events(event_name,location)

Getting Harry Styles from Salt Lake City
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.


In [138]:
def get_my_events(order_by):
    conn = create_connection()
    cur = conn.cursor()
    cur.execute(f'select name, price_low, price_high, distance, date, city, state, depart_airport, dest_airport from my_events {order_by} limit 5')
    out = cur.fetchall()
    conn.close()
    return out



In [184]:
def get_flight_search_json(origin, destination, event_date):

    d = datetime.datetime.strptime(event_date,"%Y-%m-%d")
    delta = datetime.timedelta(days=1)
    start = datetime.datetime.strftime(d - delta, "%Y-%m-%d")
    end = datetime.datetime.strftime(d + delta, "%Y-%m-%d")
    return {
              "currencyCode": "USD",
              "originDestinations": [
                {
                  "id": "1",
                  "originLocationCode": origin,
                  "destinationLocationCode": destination,
                  "departureDateTimeRange": {
                    "date": start,
                    "time": "8:00:00"
                  }
                },
                {
                  "id": "2",
                  "originLocationCode": destination,
                  "destinationLocationCode": origin,
                  "departureDateTimeRange": {
                    "date": end,
                    "time": "17:00:00"
                  }
                }
              ],
              "travelers": [
                {
                  "id": "1",
                  "travelerType": "ADULT"
                }
              ],
              "sources": [
                "GDS"
              ],
              "searchCriteria": {
                "maxFlightOffers": 1

              }
            }

In [185]:
body = {'currencyCode': 'USD', 'originDestinations': [{'id': '1', 'originLocationCode': 'SLC', 'destinationLocationCode': 'PHX', 'departureDateTimeRange': {'date': '2020-08-28', 'time': '8:00:00'}}, {'id': '2', 'originLocationCode': 'PHX', 'destinationLocationCode': 'SLC', 'departureDateTimeRange': {'date': '2020-08-30', 'time': '17:00:00'}}], 'travelers': [{'id': '1', 'travelerType': 'ADULT'}], 'sources': ['GDS'], 'searchCriteria': {'maxFlightOffers': 1}}

In [186]:
print(body)

{'currencyCode': 'USD', 'originDestinations': [{'id': '1', 'originLocationCode': 'SLC', 'destinationLocationCode': 'PHX', 'departureDateTimeRange': {'date': '2020-08-28', 'time': '8:00:00'}}, {'id': '2', 'originLocationCode': 'PHX', 'destinationLocationCode': 'SLC', 'departureDateTimeRange': {'date': '2020-08-30', 'time': '17:00:00'}}], 'travelers': [{'id': '1', 'travelerType': 'ADULT'}], 'sources': ['GDS'], 'searchCriteria': {'maxFlightOffers': 1}}


In [187]:
response = amadeus.shopping.flight_offers_search.post(body)

In [188]:
flight_results = response.data[0]

In [189]:
flight_results

{'type': 'flight-offer',
 'id': '1',
 'source': 'GDS',
 'instantTicketingRequired': False,
 'nonHomogeneous': False,
 'oneWay': False,
 'lastTicketingDate': '2020-06-02',
 'numberOfBookableSeats': 9,
 'itineraries': [{'duration': 'PT1H33M',
   'segments': [{'departure': {'iataCode': 'SLC',
      'terminal': '2',
      'at': '2020-08-28T22:15:00'},
     'arrival': {'iataCode': 'PHX',
      'terminal': '3',
      'at': '2020-08-28T22:48:00'},
     'carrierCode': 'DL',
     'number': '1192',
     'aircraft': {'code': '319'},
     'operating': {'carrierCode': 'DL'},
     'duration': 'PT1H33M',
     'id': '1',
     'numberOfStops': 0,
     'blacklistedInEU': False}]},
  {'duration': 'PT1H38M',
   'segments': [{'departure': {'iataCode': 'PHX',
      'terminal': '3',
      'at': '2020-08-30T18:45:00'},
     'arrival': {'iataCode': 'SLC',
      'terminal': '2',
      'at': '2020-08-30T21:23:00'},
     'carrierCode': 'DL',
     'number': '1449',
     'aircraft': {'code': '319'},
     'operating

In [191]:
t_flight = flight_results['itineraries'][0]['segments'][0]['departure']
r_flight = flight_results['itineraries'][1]['segments'][0]['departure']
print(f"depart:{t_flight['iataCode']} at:{t_flight['at']} ret:{r_flight['iataCode']} at:{r_flight['at']} ${flight_results['price']['grandTotal']}")

depart:SLC at:2020-08-28T22:15:00 ret:PHX at:2020-08-30T18:45:00 $206.20


In [206]:
my_events = get_my_events("order by distance, price_low, date")
with open('my_events.csv', mode='w') as events_file:
    event_writer = csv.writer(events_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    current_event = ''
    for event in my_events:
        body = get_flight_search_json(event[7], event[8], event[4])
        response = amadeus.shopping.flight_offers_search.post(body)
        flight_results = response.data[0]
        t_flight = flight_results['itineraries'][0]['segments'][0]['departure']
        r_flight = flight_results['itineraries'][1]['segments'][0]['departure']
        low = event[1] + float(flight_results['price']['grandTotal'])
        high = event[2] + float(flight_results['price']['grandTotal'])
        if current_event != event[0]:
            print(event[0])
            current_event=event[0]
        print(f"total cost:${low:.2f} to ${high:.2f} event tickets:${event[1]:.2f} to ${event[2]:.2f} flight:${flight_results['price']['grandTotal']} distance:{event[3]:.2f} date:{event[4]}, city:{event[5]}, state:{event[6]}, depart:{t_flight['iataCode']} at:{t_flight['at']} ret:{r_flight['iataCode']} at:{r_flight['at']}")
        event_writer.writerow(event)
        

Harry Styles: Love On Tour
total cost:$240.70 to $370.70 event tickets:$34.50 to $164.50 flight:$206.20 distance:501.92 date:2020-08-29, city:Glendale, state:AZ, depart:SLC at:2020-08-28T22:15:00 ret:PHX at:2020-08-30T18:45:00
total cost:$195.70 to $325.70 event tickets:$39.50 to $169.50 flight:$156.20 distance:587.07 date:2020-09-02, city:Inglewood, state:CA, depart:SLC at:2020-09-01T22:28:00 ret:ONT at:2020-09-03T18:53:00
total cost:$284.70 to $414.70 event tickets:$36.50 to $166.50 flight:$248.20 distance:995.82 date:2020-07-19, city:Saint Paul, state:MN, depart:SLC at:2020-07-18T17:27:00 ret:MSP at:2020-07-20T17:52:00
total cost:$309.70 to $459.70 event tickets:$39.50 to $189.50 flight:$270.20 distance:1002.98 date:2020-08-13, city:Dallas, state:TX, depart:SLC at:2020-08-12T20:20:00 ret:DFW at:2020-08-14T19:55:00
total cost:$315.70 to $435.70 event tickets:$39.50 to $159.50 flight:$276.20 distance:1094.05 date:2020-08-11, city:San Antonio, state:TX, depart:SLC at:2020-08-10T20:35:0